In [ ]:
from gurobipy import Model, GRB, quicksum

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
loc_x = [30,37,49,52,31,52,42,52,57,62,42,27,43,58,58,37,38,61,62,63,45,32,56]

In [ ]:
loc_y = [40,52,49,64,62,33,41,41,58,42,57,68,67,48,27,69,46,33,63,69,35,39,37]

In [ ]:
demand = [7,30,16,23,11,19,15,28,8,8,7,14,6,19,11,12,26,17,6,15,5,10]

In [ ]:
plt.plot(loc_x[0], loc_y[0], c='r', marker='s')
plt.scatter(loc_x[1:], loc_y[1:], c='b')

In [ ]:
n = 22

In [ ]:
N = [i for i in range(1, n+1)]

In [ ]:
V = [0] + N 

In [ ]:
arcs_for_savings = [(i, j) for i in V for j in V if i != j]

In [ ]:
Q = 40

In [ ]:
savings = {(i, j):    (np.hypot(loc_x[i]-loc_x[0], loc_y[i]-loc_y[0]))
                    +(np.hypot(loc_x[0]-loc_x[j], loc_y[0]-loc_y[j]))
                    -(np.hypot(loc_x[i]-loc_x[j], loc_y[i]-loc_y[j])) for i, j in arcs_for_savings}

In [ ]:
distance = {(i, j): np.hypot(loc_x[i]-loc_x[j], loc_y[i]-loc_y[j]) for i, j in arcs_for_savings}

In [ ]:
r = 8

In [ ]:
mdl = Model('CVRP')

In [ ]:
x = mdl.addVars(arcs_for_savings, vtype=GRB.BINARY)
y = mdl.addVars(N, vtype=GRB.CONTINUOUS)

In [ ]:
mdl.setObjective(quicksum(x[i, j] * savings[i, j] for  i, j in arcs_for_savings), GRB.MAXIMIZE )

In [ ]:
mdl.addConstr(quicksum(x[ 0 , j ] for j in N) == r)
mdl.addConstrs(quicksum(x[ i , j ] for i in V if i != j) == 1 for j in N)
mdl.addConstrs(quicksum(x[ i , j ] for j in N if i != j) <= 1 for i in N)
mdl.addConstrs(y[i]+ demand[i-1]*x[ i , j ]- Q * ( 1 - x[ i , j ]) <= y[j] for i in N for j in N if i != j)
mdl.addConstrs(demand[ i-1 ] <= y[ i ] for i in N)
mdl.addConstrs(y[ i ] <= Q for i in N)

In [ ]:
mdl.optimize()

In [ ]:
active_arcs = [a for a in arcs_for_savings if x[a].x > 0.99]

In [ ]:
active_arcs

In [ ]:
for i, j in active_arcs:
    plt.plot([loc_x[i], loc_x[j]], [loc_y[i], loc_y[j]], c='g', zorder=0)
plt.plot(loc_x[0], loc_y[0], c='r', marker='s')
plt.scatter(loc_x[1:], loc_y[1:], c='b')
plt.title('CVRP2_7')
plt.savefig('CVRP2_7')

In [ ]:
Total_Sum = 0
for i,j in active_arcs:
    Total_Sum = Total_Sum + distance[(i,j)]
print(Total_Sum)